# Head Counting Pipeline – 3 Aşamalı Deneysel Analiz Raporu (Jupyter Markdown)

Bu rapor, SCUT-HEAD tabanlı veri seti üzerinde yürütülen **3 aşamalı** head counting sürecinin (inference + post-process + fine-tune) nicel sonuçlarını ve teknik yorumunu sunar. Amaç, sınıf ortamındaki kişi sayısını **head detection** üzerinden güvenilir şekilde tahmin etmektir.

---

## Özet

Çalışma üç ana aşamadan oluşmaktadır:

1. **Aşama-1 (Baseline Counting / İlk Ölçüm):**  
   Modelin sayım performansının ilk değerlendirmesi (700 görüntü).  
   > Not: Aşama-1 metrikleri ile Aşama-2 metrikleri aynı tablo olarak elde edilmiştir (MAE=1.6657, RMSE=4.3668, MAPE=0.0700, GT=Pred=18605). Bu durum, ya Aşama-1 ve Aşama-2’nin aynı konfigürasyonla tekrar çalıştırıldığını ya da raporlama çıktısının tekrar kullanıldığını gösterir.

2. **Aşama-2 (Post-Processing Optimizasyonu – DIoU NMS):**  
   Conf + DIoU threshold + IoU parametreleri ile NMS tabanlı filtreleme uygulanmıştır.  
   Hedef: **False positive / duplicate** kutuları azaltırken **kaçırma** oranını yükseltmemek.

3. **Aşama-3 (Fine-tuning – Final Model):**  
   YOLOv8n modeli yeniden fine-tune edilmiştir. Ardından sayım için conf/IoU grid testi yapılmış ve en iyi konfigürasyon seçilmiştir.  
   Hedef: Sayım metriklerini proje entegrasyonuna uygun seviyeye çekmek.

---

## Aşama-1: İlk Ölçüm (Baseline)

Aşağıdaki sayım metrikleri 700 test görüntüsü üzerinde raporlanmıştır:

| Metric | Value |
|---|---:|
| MAE | 1.665714 |
| RMSE | 4.366758 |
| MAPE (ignore gt=0) | 0.070038 |
| N_images | 700 |
| GT_total | 18605 |
| Pred_total | 18605 |

### Yorum (Aşama-1)
- **GT_total ile Pred_total eşit** (18605 = 18605). Bu, toplam sayımın topluca tutturulduğunu gösterir; ancak bu tek başına yeterli değildir.
- MAE=1.6657 ve RMSE=4.3668 değerleri, görüntü bazında hatanın bazı örneklerde yüksek olabildiğini gösterir (RMSE’nin MAE’den belirgin yüksek olması → uç örneklerde hata büyüyor).
- Bu aşamada tipik risk şudur: Bazı görüntülerde **fazla sayım**, bazılarında **eksik sayım** yapıp toplamda birbirini dengelemek. Bu yüzden “toplam eşit” olmak, per-image doğruluğun garanti olduğu anlamına gelmez.

---

## Aşama-2: Post-Processing Optimizasyonu (DIoU NMS)

Aşağıdaki konfigürasyonla inference sonrası NMS uygulanmıştır:

- **conf:** 0.39  
- **nms_type:** DIoU  
- **diou_thr:** 0.51  
- **nms_iou:** 0.50  
- **imgsz:** 832  
- **device:** cuda:0  

### NMS İstatistikleri

| Alan | Değer |
|---|---:|
| n_images | 700 |
| raw_total (NMS öncesi toplam kutu) | 21598 |
| kept_total (NMS sonrası kalan kutu) | 18605 |
| removed_total (silinen kutu) | 2993 |
| avg_raw_per_image | 30.8543 |
| avg_kept_per_image | 26.5786 |
| avg_removed_per_image | 4.2757 |

### Sayım Özeti

| Metric | Value |
|---|---:|
| MAE | 1.665714 |
| RMSE | 4.366758 |
| MAPE (ignore gt=0) | 0.070038 |
| N_images | 700 |
| GT_total | 18605 |
| Pred_total | 18605 |

### Yorum (Aşama-2)
**(1) NMS etkisi sayısal olarak net:**  
- NMS öncesi 21598 kutu → NMS sonrası 18605 kutu  
- **2993 kutu silinmiş** (ortalama ~4.28 kutu/görüntü)

Bu, modelin ham çıktısında belirgin bir **duplicate / aşırı tespit** davranışı olduğunu gösterir. DIoU NMS bu fazlalığın önemli kısmını temizlemiştir.

**(2) Toplam sayımın GT ile birebir eşleşmesi:**  
- kept_total = GT_total = Pred_total = 18605  
Bu senin sayım çıktın açısından “toplam seviyede mükemmel” görünür.

**(3) Ancak hata metrikleri aynı kaldığı için kritik çıkarım:**  
- Bu konfigürasyon toplamı tutturmuş olsa da görüntü bazında hata dağılımı hâlâ geniş olabilir.
- RMSE’nin yüksek kalması, bazı görüntülerde **±5, ±10** gibi sapmaların hâlâ olduğunu düşündürür.
- Bu yüzden Aşama-2 “post-process ile dengeleme” sağlıyor; fakat “her görüntüde tutarlılık” garantisi vermez.

**(4) Teknik anlamı:**  
Aşama-2 ile asıl kazanım, modelin çıktısındaki **fazla kutuları kontrol altına almak** olmuştur. Bu aşama modelin algılama kapasitesini değiştirmez, sadece çıktıyı düzenler.

---

## Aşama-3: Fine-tuning ile Final Model

Bu aşamada model fine-tune edilmiştir ve final detection metrikleri raporlanmıştır.

### 3.1 Kullanılan Model ve Ortam

- **Backbone:** YOLOv8n (Ultralytics)
- **Task:** Detect (single-class head)
- **Framework:** Ultralytics YOLO v8.3.x
- **Device:** RTX 3060 Laptop GPU (6 GB)
- **AMP:** Açık

### 3.2 Eğitim Konfigürasyonu (Final)

| Parametre | Değer |
|---|---|
| Image Size | 896 × 896 |
| Epochs | 30 |
| Batch Size | 4 |
| Optimizer | AdamW |
| Initial LR | 0.002 |
| Weight Decay | 0.0005 |
| Mosaic | Enabled (epoch 25 sonrası kapandı) |
| Patience | 10 |
| Augmentation | RandAugment (YOLOv8 varsayılan) |

### 3.3 Detection Performansı (Val)

| Metric | Value |
|---|---:|
| Precision | 0.942 |
| Recall | 0.937 |
| mAP@50 | 0.967 |
| mAP@50–95 | 0.532 |

**Yorum (Detection):**
- Precision/Recall dengesi güçlü → hem fazla sayımı (FP) hem kaçırmayı (FN) iyi kontrol ediyor.
- mAP50 yüksek (0.967) → tespitlerin büyük kısmı doğru.
- mAP50-95 (0.532) → kutu lokasyon hassasiyeti orta-iyi seviyede (özellikle küçük ve yoğun objelerde beklenen bir durum).

### 3.4 Sayım Performansı (Final Model – En İyi Konfig)

Sayım için farklı conf/IoU eşikleri test edilmiş ve en iyi kombinasyon seçilmiştir:

- **Confidence:** 0.34  
- **IoU:** 0.45  

| Metric | Value |
|---|---:|
| N_images | 350 |
| GT_total | 9159 |
| Pred_total | 9166 |
| Abs_total_diff | 7 |
| MAE | 1.2886 |
| RMSE | 2.6560 |
| MAPE(ignore gt=0) | 0.0576 |
| Bias (Pred − GT) | +7 |

**Yorum (Counting):**
- Abs_total_diff = **7** → toplamda neredeyse birebir.
- MAE **1.2886** → görüntü başına ortalama hata ~1.29 kişi.
- RMSE **2.6560** → büyük hata yapılan uç görüntülerin sayısı azalmış (Aşama-1/2’ye göre daha iyi).
- Bias **+7** → çok hafif “fazla sayma” eğilimi; ancak pratikte ihmal edilebilir düzeyde.

---

## Aşamalar Arası Karşılaştırmalı Teknik Yorum

### 1) Aşama-2’nin gerçek katkısı: “çıktıyı temizlemek”
- Modelin ham çıktısı çok sayıda duplicate üretmiş (raw_total 21598).
- DIoU NMS ile 2993 kutu temizlenmiş.
- Toplamda GT ile eşleşme sağlanmış.
- Ancak MAE/RMSE metrikleri aynı kaldığı için, Aşama-2’nin ana etkisi:
  - **False positive / duplicate azaltma**
  - **toplamı dengeleme**
  - fakat **image-level stabiliteyi tek başına garanti etmeme**

### 2) Aşama-3’ün gerçek katkısı: “genelleme + image-level tutarlılık”
- Fine-tune sonrası mAP50 ve precision/recall yüksek.
- Sayım tarafında MAE/RMSE belirgin şekilde daha düşük seviyeye gelmiş.
- Yani artık yalnızca “toplamı tutturma” değil, **görüntü bazında hata dağılımı** da daralmış.

### 3) Neden tekrar eğitim (bir eğitim daha) çoğu durumda gereksiz?
- Eğitim eğrileri plato yapıyorsa, ek epoch çoğunlukla marjinal kazanç getirir.
- Mevcut noktada en büyük kazanım, inference tarafında conf/IoU/NMS optimizasyonu ile sağlanmıştır.
- Eğer hâlâ bazı spesifik görüntü türlerinde (aşırı kalabalık, aşırı uzak, motion blur) sapmalar varsa, çözüm:
  - yeni veri toplama / domain çeşitliliği
  - ya da hard-case odaklı ek fine-tune
  olacaktır.

---

## Nihai Çıktılar ve Dosya Konumu

Final model ağırlıkları:

```text
C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\third_stage\runs\best_finetune_img896_ep30.pt


-----
## Sonuç

 Bu 3 aşamalı yaklaşım sonucunda:

* Aşama-2, duplicate/FP temizliği ile çıktıyı stabilize etmiş, toplam sayımı dengelemiştir.

* Aşama-3, modelin genel tespit gücünü ve sayımın görüntü bazındaki tutarlılığını artırmıştır.

- Final sistem, sınıf ortamı kişi sayımı senaryosu için yüksek doğruluklu ve entegrasyona uygun seviyeye getirilmiştir.